<a href="https://colab.research.google.com/github/nyoons/ESAA/blob/main/2022_10_31_%EA%B3%BC%EC%A0%9C_%ED%8C%8C%EC%9D%B4%EC%8D%AC_%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D_%EC%99%84%EB%B2%BD%EA%B0%80%EC%9D%B4%EB%93%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ch 03. 평가
머신러닝의 프로세스는 데이터 가공-모델 학습과 예측-평가. 성능평가지표는 모델이 분류인지 회귀인지에 따라 여러 종류로 나뉜다. 특히 분류의 성능평가지표로는 정확도, 오차행렬, 정밀도, 재현율, F1 스코어, ROC AUC 등이 있다. 이진/멀티분류에 모두 적용할 수 있지만 이진분류에서 더욱 강조하는 지표.
>**01.정확도**

=예측 결과가 동일한 데이터 건수 / 전체 예측 데이터 건수
<br/>데이터의 구성에 따라 모델의 성능을 왜곡할 수 있어서 수치 하나로는 평가하지 않음. 어떻게 왜곡?
<br/>앞의 타이타닉 고려. ml알고리즘 적용 후 예측 정확도는 보통 80퍼대였지만, 탑승객이 남자인 경우보다 여자인 경우 생존 확률이 높았으니 별다른 알고리즘 없이 무조건 여자인 경우 생존, 남자인 경우 사망으로 예측해도 비슷한 수치가 나올수도. 성별 조건 하나만으로 결정하는 별거 아닌 알고리즘도 정확도가 높아짐.
<br/>BaseEstimator 받아 아무런 학습 없이 성별에 따라 예측하는 단순 classifier 생성해보자

In [ ]:
from sklearn.base import BaseEstimator

class MyDummyClassifier(BaseEstimator):
  #fit메서드는 아무것도 학습하지 않게
  def fit(self, X, y=None):
    pass
  #predict() 메서드는 단순히 sex 피처가 1이면 0, 아니면 1로 예측
  def predict(self, X):
    pred=np.zeros((X.shape[0],1))
    for i in range(X.shape[0]) :
      if X['Sex'].iloc[i]==1:
        pred[i]=0
      else :
        pred[i]=1

    return pred

생성된 MyDummyClassifier 이용해 타이타닉 생존자 예측 해보자

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

#원본 데이터 재로딩, 데이터 가공, 학습 데이터/테스트 데이터로 분할
titanic_df=pd.read_csv('titanic_train.csv')
y_titanic_df=titanic_df['Survived']
X_titanic_df=titanic_df.drop('Survived', axis=1)
X_titanic_df=transform_features(X_titanic_df)
X_train, X_test, y_train, y_test=train_test_split(X_titanic_df, y_titanic_df, test_size=0.2, random_state=0)

#위에서 생선한 Dummy Classifier 이용해 학습/예측/평가 수행
myclf=MyDummyClassifier()
myclf.fit(X_train, y_train)

mypredictions=myclf.predict(X_test)
print('Dummy Classfier의 정확도는: {0:.4f}'.format(accuracy_score(y_test, mypredictions)))

NameError: ignored

이렇게만 해도 정확도가 0.7877. 정확도는 불균형한 레이블 값 분포에서 적합하지 않음.
<br/>MNIST로도 보자. 7만 True, 아닌건 False로 변환해 이진 분류로 바꿔 예측하자.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd

class MyDummyClassifier(BaseEstimator):
  def fit(self, X, y):
    pass

  #입력값으로 들어오는 X 데이터 세트 크기만큼 모두 0값으로 만들어 반환
  def predict(self, X):
      return np.zeros( (len(X), 1), dtype=bool)

#사이킷런의 내장 데이터 세트인 load_digits 이용해 MNIST 데이터 로딩
digits=load_digits()

#digits 번호가 7이면 TRUE 이고 이를 astype(int)로 1로 변환, 7 아니면 False 이고 0으로 변환.
y=(digits.target==7).astype(int)
X_train, X_test, y_train, y_test=train_test_split(digits.data, y, random_state=11)


NameError: ignored

불균형한 데이터로 생성한 y_test의 분포 확인, 예측/평가 수행

In [ ]:
#불균형한 레이블 데이터 분포 확인
print('레이블 테스트 세트 크기 :', y_test.shape)
print('테스트 세트 레이블 0과 1의 분포도 : ')
print(pd.Series(y_test).value_counts())

#Dummy Classifier로 학습/예측 정확도 평가
fakeclf=MyFakeClassifier()
fakeclf.fit(X_train, y_train)
fakepred=fakeclf.predict(X_test)
print('모든 예측을 0으로 하여도 정확도는:{:.3f}'.format(accuracu_score(y_test,fakepred)))

NameError: ignored

이렇게만 해도 정확도가 90퍼. 정확도는 불균형한 레이블 데이터 세트에서는 사용하면 안됨.

>**2. 오차 행렬**

학습된 분류 모델이 예측을 수행하며 얼마나 헷갈리고 있는지 보여주는 지표. TN, FP, FN, TP를 생각하자.
<br/>사이킷런은 오차 행렬을 구하기 위해 confusion_matrix() 제공

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, fakepred)


NameError: ignored

ndarray 형태로 출력. TP, TN, FN, FP를 보면 정확도가 왜 오류를 일으키는지 알 수 있다.

> **3. 정밀도와 재현율**

positive 데이터 세트의 예측 성능에 초점을 맞춘 평가 지표. 앞에서 만든 myfakeclf는 positive로 예측한 TP 값이 하나도 없어서 정밀도와 재현율이 모두 0.
<br/>정밀도=TP/(FP+TP)
<br/>재현율=TP/(FN+TP) = 민감도, TPR
<br/>재현율이 중요 지표인 경우는 실제 positive 양성 데이터를 negative로 잘못 판단하게 되면 업무상 큰 영향이 발생하는 경우. 보험사기나 금융사기 적발 모델, 암 판단 모델 등.
<br/>정밀도가 중요한 지표인 경우는 스패메일 여부를 판단하는 모델. 실제 negative 음성인 데이터 예측을 positive 양성으로 잘못 판단하면 업무상 큰 영향이 발생하는 경우.
<br/>사이킷런은 정밀도 계산을 위해 precision_score(), 재현율 계산을 위해 recall_score() 제공

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
def get_clf_eval(y_test, pred):
  confusion=confusion_matrix(y_test, pred)
  accuracy=precision_score(y_test, pred)
  precision=precision_score(y_test, pred)
  recall=recall_score(y_test, pred)
  print('오차 행렬')
  print(confusion)
  print('정확도:{0:.4f}, 재현율:{2:.4f}'.format(accuracy, precision, recall))


In [ ]:
#이제 로지스틱 회귀 기반으로 타이타닉 예측하고 평가 수행.
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

#원본 데이터 재로딩, 데이터 가공, 학습 데이터/테스트 데이터 분할
titanic_df=pd.read_csv('titanic_train.csv')
y_titanic_df=titanic_df['Survived']
X_titanic_df=titanic_df.drop('Survived', axis=1)
X_titanic_df=transform_features(X_titanic_df)

X_train, X_test, y_train, y_test=train_test_split(X_titanic_df, y_titanic_df, test_size=0.20, random_state=11)

lr_clf=LogisticRegression()
lr_clf.fit(X_train, y_train)
pred=lr_clf.predict(X_test)
get_clf_eval(y_test,pred)

FileNotFoundError: ignored

재현율이 좀 낮게 나왔다. 강화할 방법?

###정밀도/재현율 트레이드오프

업무 특성에 따라 정밀도나 재현율이 강조돼야 할 경우 분류의 결정 임계값을 조정해 수치를 높일 수 있다. 하지만 둘은 상호 보완적.
<br/>사이킷런의 분류 알고리즘은 예측 데이터가 특정 레이블에 속하는지 계산하기 위해먼저 개별 레이블별 결정 확률을 구하고, 큰 예측 확률로 예측. 이진 분류에서 임계값은 0.5. predict_proba() 제공. 


In [ ]:
pred_proba=lr_clf.predict_proba(X_test)
pred=lr_clf.predict(X_test)
print('pred_proba()결과 Shape:{0}'.format(pred_proba.shape))
print('pred_proba array에서 앞 3개만 샘플로 추출 \n:', pred_proba[:3])

#예측 확률 array와 예측 결과값 array를 병합해 예측 확률과 결과값을 한눈에 확인
pred_proba_result=np.concatenate([pred_proba, pred.reshape(-1,1)], axis=1)
print('두 개의 class 중에서 더 큰 확률을 클래스 값으로 예측 \n', pred_proba_result[:3])

반환 결과의 ndarray는 0과 1에 대한 확률. 더 큰 값으로 예측.
<br/>predict 매서드가 predict_proba 메서드에 기반된 것. 이제 트레이드오프 이해해보자.
<br/>사이킷런의 Binarizer 클래스를 이용. 임계값 변수를 특정 값으로 설정하고 binarizer 클래스를 객체로 생성해보자.

In [1]:
from sklearn.preprocessing import Binarizer
X=[[1,-1,2],
   [2,0,0],
   [0,1.1,1.2]]
#X의 개별 원소들이 임계값보다 작거나 같으면 0, 아니면 1 반환
binarizer=Binarizer(threshold=1.1)
print(binarizer.fit_transform(X))

[[0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]]


이제 이 binarizer 이용해 predict()의 의사 코드를 만들어보자

In [ ]:
from sklearn.preprocessing import Binarizer
#Binarizer이 임계값 설정값. 분류 결정 임계값임.
custom_threshold=0.5

#predict_proba 반환값의 두번째 칼럼, 즉 positive 클래스 칼럼 하나만 추출해 적용
pred_proba_1=pred_proba[:,1].reshape(-1,1)

binarizer=Binarizer(threshold=custom_threshold).fit(pred_proba_1)
custom_predict=binarizer.transform(pred_proba_1)

get_clf_eval(y_test, custom_predict)


오차 행렬 도출됨. 임계값을 낮추면?

In [ ]:
custom_threshold=0.4

#predict_proba 반환값의 두번째 칼럼, 즉 positive 클래스 칼럼 하나만 추출해 적용
pred_proba_1=pred_proba[:,1].reshape(-1,1)

binarizer=Binarizer(threshold=custom_threshold).fit(pred_proba_1)
custom_predict=binarizer.transform(pred_proba_1)

get_clf_eval(y_test, custom_predict)


재현율 올라가고 정밀도 감소. 분류 결저 임계값은 positive 예측값을 결정하는 확률의 기준이 된다. 예측을 너그럽게 하게 되면 True 값이 많아진다. 그렇다면 재현율이 높아지지.
<br/>임계값을 0.4에서 0.6까지 0.05씩 증가시키며 지표 조사하자.

In [ ]:
#테스트를 수행할 모든 임계값을 리스트 객체로 저장.
thresholds=[0.4,0.45,0.5,0.55,0.6]

def get_eval_by_threshold(y_test,pred_proba_c1,thresholds):
  #thresholds list 객체 내의 값 차례로 반복하며 평가 수행
  for custom_threshold in thresholds:
    binarizer=Binarizer(threshold=custom_threshold).fit(pred_proba_c1)
    custom_predict=binarizer.transform(pred_proba_c1)
    print('임계값:', custom_threshold)
    get_clf_eval(y_test, custom_predict)

get_eval_by_threshold(y_test, pred_proba[:1].reshape(-1,1), thresholds)

0.45일 경우 0.5에 비해 정밀도는 약간 떨어지나 재현율이 오름. 정확도는 동일. 재현율을 향상시키려면 0.45가 적당.
<br/>이렇게 임계값 변화에 따른 평가 지표 값을 알아보려면 사이킷런의 precision_recall_curve() 사용.
<br/>타이타닐 적용해보자. 인자로 실제 값 데이터 세트와 레이블 값이 1일 때의 예측 확률을 입력.

In [ ]:
from sklearn.metrics import precision_recall_curve

#레이블값이 1인 경우 예측 확률 추출
pred_proba_class1=lr_clf.predict_proba(X_test)[:,1]

#실제 값 데이터 세트와 레이블 값이 1일 때의 예측 확률을 precision_recall_curve 인자로 입력
precisions, recalls, thresholds=precision_recall_curve(y_test,pred_proba_class1)
print('반환된 분류 결정 임계값 배열의 shape:', thresholds.shape)

#반환된 임계값 배열 로우가 147건이므로 샘플로 10건만 추출, 임계값을 15step으로 추출.
thr_idx=np.arange(0, thresholds.shape[0], 15)
print('샘플 추출을 위한 임계값 배열의 index 10개:', thr_idx)
print('샘플용 10개의 임계값:', np.round(thresholds[thr_idx],2))

#15step 단위로 추출된 임계값에 따른 정밀도와 재현율 값
print('샘플 임계값 별 정밀도:', np.round(precisions[thr_idx],3))
print('샘플 임계값 별 재현율:', np.round(recalls[thr_idx], 3))

임계값 증가할수록 정밀도 높아지나 재현율은 낮아짐. precison_recall_curve() 사용해 재현율의 변화를 곡선으로 시각화하자.

In [ ]:
from pandas.errors import InvalidIndexError
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
%matplotlib inline

def precision_recall_curve_plot(y_test, pred_proba_c1):
  #threshold ndarray와 이 threshold에 따른 정밀도, 재현율 ndarray 추출.
  precisions, recalls, thresholds = precison_recall_curve(y_test, pred_proba_c1)

  #X출을 threshold 값으로, y축은 정밀도, 재현율 값으로 각각 plot 수행. 정밀도는 점선으로 표시
  plt.figure(figsize=(8,6))
  threshold_boundary=thresholds.shape[0]
  plt.plot(thresholds, precisions[0:threshold_boundary], linestyle='-', label='precision')
  plt.plot(thresholds, recalls[0:threshold_boundary], label='recall')

  #thresholds 값 x축의 scale을 0.1 단위로 변경
  start, end=plt.xlim()
  plt.xticks(np.round(np.arange(start, end, 0.1),2))

  #x,y축 label과 legend, grid 설정
  plt.xlabel('Threshold value'); plt.ylabel('Precision and Recall curve')
  plt.legend(); plt.grid()
  plt.show()

precision_recall_curve_plot(y_test, lr_clf.predict_proba(X_test)[:,1])

역시 임계값이 낮을수록 재현율이 높고 정밀도가 낮다. 

###정밀도와 재현율의 맹점
상호 보완적.

#### 정밀도가 100%가 되는 방법
확실한 기준이 되는 경우만 positive, 나머지는 모두 negative로 예측
#### 재현율이 100%가 되는 방법
모든 환자를 positive로 예측.

이처럼 극단적 수치 조작 가능... 그러나 균형 있게 사용해야함

<br/>

>**4. F1 스코어**

정밀도와 재현율을 결합한 지표. 어느 한쪽으로 치우치지 않을 때 높은 수치. 사이킷런의 f1_score() 이용.

In [ ]:
from sklearn.metrics import f1_score
f1=f1_score(y_test, pred)
print('F1 스코어: {0:.4f}'.format(f1))

타이타닉에서 임계값 변화시키며 f1 스코어 등 평가 지표 구해보자.

In [ ]:
def get_clf_eval(y_test, pred):
  confusion=confusion_matrix(y_test, pred)
  accuracy=accuracy_score(y_test, pred)
  precision=precision_score(y_test, pred)
  recall=recall_score(y_test, pred)
  #F1 스코어 추가
  f1=f1_score(y_test, pred)
  print('오차 행렬')
  print(confusion)
  #f1 score print 추가
  print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f}, F1:{3:.4f}'.format(accuracy, precision, recall, f1))

thresholds=[0.4,0.45,0.5,0.55,0.6]
pred_proba=lr_clf.predict_proba(X_test)
get_eval_by_threshold(y_test, pred_proba[:,1].reshape(-1,1), thresholds)

f1 스코어는 임계값이 0.6일때 가장 좋다.

>**5. ROC 곡선과 AUC**

ROC 곡선은 FPR이 변할 때 TPR이 어떻게 변하는가. 
<br/>ROC 곡선은 FPR을 0부터 1까지 변화시키며 TPR의 값을 구한다. 임계값을 변경하며 수행하면 된다. 임계값=1이면 FPR=0, 임계값=0이면 FPR=1

In [ ]:
from sklearn.metrics import roc_curve

#레이블 값이 1일 때의 예측 확률 추출
pred_proba_class1=lr_clf.predict_proba(X_test)[:,1]

fprs, tprs, thresholds=roc_curve(y_test, pred_proba_class1)
#반환된 임계값 배열 로우가 47건이므로 샘플로 10건만 추출, 임계값을 5step으로 추출.
#thresholds[0]은 max(예측확률)+1로 임의 설정. 제외하기 위해 np.arange는 1부터 시작.
thr_index=np.arange(1,thresholds.shape[0], 5)

print('샘플 추출을 위한 임계값 배열의 index 10개:', thr_index)
print('샘플용 10개의 임계값:', np.round(thresholds[thr_index],2))

#5step 단위로 추출된 임계값에 따른 FPR, TPR 값
print('샘플 임계값 별 FPR:', np.round(fprs[thr_index], 3))
print('샘플 임계값 별 TPR:', np.round(tprs[thr_index], 3))

임계값이 1이 가까운 값에서 점점 작아지며 FPR이 커진다. 그러면서 TPR은 가파르게 증가. 시각화하면 더 잘 보인다!
<br/>ROC 곡선 자체는 FPR과 TPR의 변화 값을 보는데 이용하고 성능 지표로 사용되는건 AUC (면적).

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score
from sklearn.metrics import recall_score, f1_score, roc_auc_score
import numpy as np

print(confusion_matrix(y_target, preds))
print('정확도:', np.round(accuracy_score(y_target, preds), 4))
print('정밀도:', np.round(precision_score(y_target, preds), 4))
print('재현율:', np.round(recall_score(y_target, preds), 4))

AUC 값은 약 0.8987. 마지막으로 get_clf_eval에 roc_auc_score 이용해 추가하자.

In [ ]:
def get_clf_eval(y_test, pred=None, pred_proba=None):
  confusion=confusion_matrix(y_test, pred)
  accuracy=accuracy_score(y_test, pred)
  precision=precision_score(y_test, pred)
  recall=recall_score(y_test, pred)
  f1=f1_score(y_test, pred)
  roc_auc=roc_auc_score(y_test, pred_proba)
  print('오차 행렬')
  print(confusion)
  print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f},\
       F1:{3:.4f}, AUC:{4:.4f}'.format(accuracy, precision, recall, f1, roc_auc))